In [386]:
from yahoofinancials import YahooFinancials
import numpy as np

In [387]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        print(gr)
        return np.size(bv)-1, np.median(gr)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def get_BV_GR_mean(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        print(gr)
        return np.size(bv)-1, np.mean(gr)
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        print(self._stock_summary_data('trailingPE'))
        print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print(suggestPrice)
        print(stockprice)
        if suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [388]:
x = cookFinancials('ZM')

In [389]:
bv = x.get_BV(20)

In [390]:
bv.insert(0, x.get_book_value())
print(bv)

[1499918000, 833943000, -7439000, -26671000, -28838000]


In [391]:
bvgr = x.get_BV_GR_median(bv)
print(bvgr)

[0.7985857546618894, 113.10418066944482, 0.7210828240410934, 0.07514390734447604]
(4, 0.7598342893514913)


In [392]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 8;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

287.76233
140.84694


In [393]:
print(x.get_earnings_per_share())


1.4390000247773915


In [394]:
x.get_pe_ratio()
x.get_PE()

287.76233
140.84694


214.30463500000002

In [395]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

1.4390000247773915
0.7598342893514913
8
0.2
214.30463500000002
0.5


In [396]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(6598.065165503826, 3299.032582751913)


In [397]:
x.get_decision(price[1],x.get_current_price())

3299.032582751913
414.09


'strong buy'